<a href="https://colab.research.google.com/github/goguaD/finalProjectML/blob/main/RandomForest" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch torchvision torchaudio
!pip install wandb
!pip install kaggle

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 71.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 106.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"ditogogua","key":"33ff556f8487784b1a2aa019105cd547"}'}

In [3]:
!pip install -q kaggle

In [4]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle competitions download -c walmart-recruiting-store-sales-forecasting

  0% 0.00/2.70M [00:00<?, ?B/s]
100% 2.70M/2.70M [00:00<00:00, 631MB/s]


In [6]:
!unzip -q walmart-recruiting-store-sales-forecasting.zip -d data/

In [7]:
!ls data

features.csv.zip	  stores.csv	train.csv.zip
sampleSubmission.csv.zip  test.csv.zip


In [8]:
!pip install -q wandb

In [9]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: dgogu22 (dgogu22-free-university-of-tbilisi-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.preprocessing import LabelEncoder

In [11]:
features = pd.read_csv("data/features.csv.zip")
train = pd.read_csv("data/train.csv.zip")
test = pd.read_csv("data/test.csv.zip")
stores = pd.read_csv("data/stores.csv")

In [12]:
train = train.merge(features, on=["Store", "Date"], how="left")
train = train.merge(stores, on="Store", how="left")
test = test.merge(features, on=["Store", "Date"], how="left")
test = test.merge(stores, on="Store", how="left")

In [13]:
for df in [train, test]:
    df["Date"] = pd.to_datetime(df["Date"])
    df["Year"] = df["Date"].dt.year
    df["Month"] = df["Date"].dt.month
    df["Week"] = df["Date"].dt.isocalendar().week.astype(int)

In [14]:
markdown_cols = ["MarkDown1", "MarkDown2", "MarkDown3", "MarkDown4", "MarkDown5"]
for col in markdown_cols:
    train[col].fillna(0, inplace=True)
    test[col].fillna(0, inplace=True)


/tmp/ipython-input-14-3178086063.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train[col].fillna(0, inplace=True)
/tmp/ipython-input-14-3178086063.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.

In [15]:
le = LabelEncoder()
train["Type"] = le.fit_transform(train["Type"])
test["Type"] = le.transform(test["Type"])


In [16]:
train.rename(columns={"IsHoliday_x": "IsHoliday", "IsHoliday_y": "IsHoliday_feat"}, inplace=True)
test.rename(columns={"IsHoliday_x": "IsHoliday", "IsHoliday_y": "IsHoliday_feat"}, inplace=True)


In [17]:
drop_cols_train = ["Date", "IsHoliday", "IsHoliday_feat", "Weekly_Sales"]
drop_cols_test = ["Date", "IsHoliday", "IsHoliday_feat"]

In [18]:

X = train.drop(columns=drop_cols_train)
y = train["Weekly_Sales"]
X_test = test.drop(columns=drop_cols_test)

In [19]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [21]:
model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
model.fit(X_train, y_train)

RandomForestRegressor(n_jobs=-1, random_state=42)

In [20]:
def weighted_mae(y_true, y_pred, is_holiday):
    weights = np.where(is_holiday, 5, 1)
    return np.round(np.sum(weights * np.abs(y_true - y_pred)) / np.sum(weights), 4)

In [24]:
# Step 10. Evaluate with WMAE
# We need to get IsHoliday values for the validation rows
val_idx = X_val.index
is_holiday_val = train.loc[val_idx, "IsHoliday"]

y_pred = model.predict(X_val)
wmae = weighted_mae(y_val, y_pred, is_holiday_val)
print("Validation WMAE:", wmae)


Validation WMAE: 1660.2124


In [25]:
model.fit(X, y)

RandomForestRegressor(n_jobs=-1, random_state=42)

In [26]:
y_train_pred = model.predict(X)
is_holiday_full = train["IsHoliday"]
wmae_full = weighted_mae(y, y_train_pred, is_holiday_full)
print("Full Training WMAE:", wmae_full)

Full Training WMAE: 585.9439


In [27]:
predictions = model.predict(X_test)

In [28]:
print(predictions)

[32245.6233 19269.2962 19761.2214 ...  1152.7848  1105.4049  1105.8849]


In [23]:
!pip install mlflow



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 93.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 90.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.7/242.7 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 733.8/733.8 kB 53.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.2/196.2 kB 21.2 MB/s eta 0:00:00


In [24]:
import mlflow

In [26]:
model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)

In [33]:
import os

os.environ["MLFLOW_TRACKING_USERNAME"] = "goguaD"  # your DagsHub username
os.environ["MLFLOW_TRACKING_PASSWORD"] = "685c4f5b2a0c555f9136c60a8666661d952de9be"  # paste your token here


In [34]:
mlflow.set_tracking_uri("https://dagshub.com/goguaD/finalProjectML.mlflow")
mlflow.set_experiment("walmart-sales")

2025/07/06 17:05:37 INFO mlflow.tracking.fluent: Experiment with name 'walmart-sales' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/0df2743fae1f428cbe1ef62bd56be3ce', creation_time=1751821538135, experiment_id='0', last_update_time=1751821538135, lifecycle_stage='active', name='walmart-sales', tags={}>

In [39]:
with mlflow.start_run():
    model.fit(X_train, y_train)

    y_pred = model.predict(X_val)
    val_idx = X_val.index
    is_holiday_val = train.loc[val_idx, "IsHoliday"]
    wmae = weighted_mae(y_val, y_pred, is_holiday_val)
    print("Validation WMAE:", wmae)
    mlflow.log_metric("RFR", wmae)
    mlflow.end_run()
    mlflow.log_param("model", "RandomForest")
    mlflow.log_param("n_estimators", 100)
    mlflow.log_param("train_split", 0.8)

Validation WMAE: 1660.2124
🏃 View run casual-lamb-218 at: https://dagshub.com/goguaD/finalProjectML.mlflow/#/experiments/0/runs/2b0bc51ff4d84ba18faeb0375b2567f6
🧪 View experiment at: https://dagshub.com/goguaD/finalProjectML.mlflow/#/experiments/0
